In [3]:
import glob

In [4]:
%%capture
!gdown https://drive.google.com/uc?id=1JuawXQmYVkjpfL3H0blqjDrqw8V1lHrC
!unrar e /content/FIRE_Dataset_EN_2010.rar
!gunzip /content/en.qrels.76-125.2010.txt.gz
!tar -xzvf English-Data.tgz

In [5]:
e = open("/content/en.qrels.76-125.2010.txt")
q = open("/content/en.topics.76-125.2010.txt")

In [6]:
estr = e.read()
qstr = q.read()

In [7]:
import numpy as np
import pandas as pd
import os
from bs4 import BeautifulSoup
import nltk

In [8]:

elist = estr.split("\n")
# elist = [est for est in es if est.endswith('1')]

edir = {}
for e in elist[:-1]:
  a, b, c, d = e.split()
  if a not in edir:
    edir[a] = [(c, d)]
  else:
    edir[a].append((c, d))


In [10]:
import re
ls = qstr.split('</top>')[:-1]
Queries={}
for query in ls:
  soup = BeautifulSoup(query, 'lxml')
  n = soup.find('num').text
  s = soup.find('desc').text
  s += soup.find('title').text
  # s = soup.find('narr').text
  
  Queries[n] = s
Queries

{'100': '\nMonica Bedi charged with obtaining forged passports at\nHyderabad.\nMonica Bedi and the passport forgery case',
 '101': "\nDetails of the drink-and-drugs party at the late Pramod\nMahajan's official bungalow in which Rahul (his son), Bibek\nMoitra, and possibly others were involved.\nDrug party at Pramod Mahajan's bungalow",
 '102': '\nCharges of doping against Shoaib Akhtar and Mohammad Asif.\nPakistani cricketers involved in a doping scandal',
 '103': "\nWhat is India's position on the Indo-Pak conflict over the\nBaglihar hydro-electric power project?\nBilateral problems surrounding the Baglihar hydro-electric\npower project",
 '104': '\nWhat legal steps has Jaya Bachchan taken on being\ndisqualified from Rajya Sabha membership for holding an\noffice of profit?\nJaya Bachchan sacked from Rajya Sabha membership',
 '105': '\nCBI investigations into the alleged involvement of the Chief\nMinister and highly placed bureaucrats of Uttar Pradesh in\nthe multi-crore Taj heritage c

In [50]:
def getdocFiles():
  files = {}
  for file in glob.iglob("TELEGRAPH_UTF8/**/*.utf8", recursive=True):
    files[file.split('/')[-1]] = open(file)
  return files

In [51]:
files = getdocFiles()

In [52]:
class doc:
  def __init__(self, docid, docobj):
    """
    docobj should be IOTextwrapper
    """
    self.docid = docid
    self.docobj = docobj

  def read(self):
    return self.docobj.read()




In [53]:
docs = [doc(id, obj) for id, obj in files.items()]

In [19]:
class corpus:
  def __iter__(self, docs):
    for doc in docs:
      yield doc.read()

In [20]:
def count():
  i = 0 
  while True:
    yield i
    i+=1


In [21]:
corps = corpus()

In [54]:
class RochioAlgorithm:
  """
  Custom Implementation of rochio algorithm

  Parameters: 
  alpha, beta, gamma

  """
  def __init__(self, Queries, KnownRel, docs):
    """
    Queries : dictonary containing ids and queries in format {id:Query}
    KnownRel : dictonary containing ids docid relevance score(0 or 1) in format

      {id: [
        (docid, relevancescore), ...
      ]
      ...}
    docs : list of doc objects

    """
    self.Queries = Queries
    self.KnownRel = KnownRel
    self.OpQueries = {}
    self.docs = docs

  def iter(self):
    """
    iterator to yield docs and queries one by one
    """
    for doc in self.docs:
      yield doc.read()
    for queryid in self.Queries:
      yield self.Queries[queryid]
  

  def nRelDocs(self, queryid):
    """
    Returns number of relevant docs for a query with queryid
    """
    return sum([b == '1' for a, b in self.KnownRel[queryid]])
  

  def nIrrDocs(self, queryid):
    """
    Returns number of Irrelevant docs for a query with queryid
    """
    return sum([b=='0' for a, b in self.KnownRel[queryid]])

  def vectorizer(self, fn):
    """
    tfidf vectorizer calculates tfidf vectors for docs and queries
    """
    itr = self.iter()
    termdocMat = fn(self.docs, token_pattern='[a-zA-Z]+', stop_words={'english'}).fit_transform(itr)
    self.tfidfvec = {}
    
    for i in range(len(self.docs)):
      self.tfidfvec[self.docs[i].docid] = termdocMat[i, :]
    
    c = count()
    self.tfidfq = {}
    for queryid in self.Queries:
      self.tfidfq[queryid] = termdocMat[len(self.docs) + next(c), :]
    del termdocMat

  def word2vecVectorizer(self, fn, files):
    """
    word2vec vectorizer calculates word2vec representation(sums w2vec vectors for each word) for docs and queries
    """

    from gensim.test.utils import datapath
    from gensim import utils
    from gensim.parsing.preprocessing import preprocess_string, preprocess_documents

    class MyCorpus:
      """An iterator that yields sentences (lists of str)."""

      def __iter__(self):
        for file in files:
          yield preprocess_string(files[file].read())

    sentences = MyCorpus()

    import gensim.models

    skipmodel = gensim.models.Word2Vec(sentences=sentences, sg=1, size=300, window=5, workers=2)
    
    docs = {file:preprocess_string(files[file]) for file in files}

    queries = {query:preprocess_string(self.Queries[query]) for query in self.Queries}

    words=[]
    for wrds in docs.values():
      words.extend(wrds)
    wordSet = set(words)
    self.docsV = {doc:sum([skipmodel.wv[word] for word in docs[doc] if word in wordSet and word in model.wv]) for doc in docs}
    self.w2vQueries = {query:sum([model.wv[word] for word in queries[id] if word in wordSet and word in model.wv]) for query in queries}

    






  def sumRelIrrDocs(self, docsV, queryID, rel):
    """
    returns the sum of docs tfidfvec
    rel : 0 for irrelevant docs 1 for relevant docs
    """
    flag = 0
    for docid, relscore in self.KnownRel[queryID]:
      if relscore == rel:
        if flag == 0:
          summ = docsV[docid]
          flag = 1
        else:
          summ += docsV[docid]
    return summ

  def computeOpQuery(self, docsV, queriesV,queryid, alpha, beta, gamma):
    """
    Computes optimized query vector representation for vector representation of query with queryid 
    """
    return alpha * queriesV[queryid] + beta * self.sumRelIrrDocs(docsV, queryid, 
          '1')/self.nRelDocs(queryid) - gamma * self.sumRelIrrDocs(docsV, queryid, '0')/self.nIrrDocs(queryid)
    

  def getOptimizedQueries(self, docsV, queriesV, alpha, beta, gamma):
    """
    computes optimized queries for all queries
    """
    for queryid in self.Queries:
      self.OpQueries[queryid] = self.computeOpQuery(docsV, queriesV ,queryid, alpha, beta, gamma)
    
  
  
        



  


In [23]:
def count():
  """
  simple counter
  """
  i = 0 
  while True:
    yield i
    i+=1
c= count()

# fileIndexer for docs
fileIndexer = {file:next(c) for file in files}

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
def cosineSim(a, b):
  """
  returns cosine similarity between two sparse vectors
  """
  return cosine_similarity(a, b)

In [25]:
class Retrieval:
  """
  Custom Implementation for performing retrieval task
  """
  def __init__(self, rochiobj, docV):
    self.OpQueries = rochiobj.OpQueries
    self.docRep = docV
    self.fileIndexer = fileIndexer
    self.KnownQRel = {query:[] for query in rochiobj.KnownRel}
    for query in rochiobj.KnownRel:
      for lst in rochiobj.KnownRel[query]:
        if lst[-1] == '1':
          self.KnownQRel[query].append(lst)
    
  
  def Similarity(self, simFunc):
    """
    calculates similarity between all possible pairs of docs and queries using similarity function passed
    """
    self.docmeas = {}
    for query in self.OpQueries:
      print(next(c1), end=',')
      ls = []
      for doc in self.docRep:
        ls.append((query, doc, simFunc(self.docRep[doc], self.OpQueries[query])))
      self.docmeas[query] = ls  

  def Retrieve(self):
    """
    sorts the docs in decreasing order of similarity score and chops off undesired one's
    """
    #Sorting in decreasing order of similarity score
    for query in self.docmeas:
      self.docmeas[query].sort(key=lambda x: x[2][0][0], reverse=True)
    
    for query in self.OpQueries:
      self.docmeas[query] = self.docmeas[query][:10]
    
  def getMap(self):
    """
    returns the map score
    """
    # self.Similarity(cosineSim)
    self.Retrieve()
    docmeas1 = {}
    for query in self.OpQueries:
      docmeas1[query] = [(c[1], 1 if isin(c[1], self.KnownQRel[query]) else 0) for c in self.docmeas[query]]
    queryap = {}
    for query in self.OpQueries:
      # print(query)
      gtp = len(self.KnownQRel[query])
      p = 0
      i = 0
      for j in range(len(docmeas1[query])):
        # print(docmeas1[query][j][1])
        if docmeas1[query][j][1] == 1:
    
          i += 1
          p += i/(j+1)
      if i == 0:
        queryap[query] = 0
      else:
        queryap[query] = p/i
    return sum(queryap.values())/ len(queryap)
  


# TFIDF


In [17]:
r = RochioAlgorithm(Queries, edir, docs)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
r.vectorizer(TfidfVectorizer)

In [20]:
r.getOptimizedQueries(1, 0.75, 0.15)

In [156]:
c1 = count() 

In [163]:
def isin(el, obj):
  for x in obj:
    if x[0] == el:
      return True
  return False

In [164]:
ret = Retrieval(r, r.tfidfvec)

In [165]:
ret.docmeas = obj

In [166]:
ret.getMap()

0.7463174358802956

Copied similarity scores to obj to use it later

In [161]:
# obj = ret.docmeas.copy()

# word2vec


In [ ]:
files = getdocFiles()

In [ ]:
r.word2vecVectorizer(files)

In [ ]:
r.getOptimizedQueries(r.docsV, r.w2vQueries, 1, 0.75, 0.15)

In [ ]:
def count():
  i = 0 
  while True:
    yield i
    i+=1
c= count()
fileIndexer = {file:next(c) for file in files}

In [ ]:
retw2v = Retrieval(r, r.docsV)

In [ ]:
def cosineSim1(a, b):
  an = np.linalg.norm(a)
  bn = np.linalg.norm(b)
  if an == 0 or bn == 0:
    return 0

  return np.dot(a, b)/(an*bn)

In [ ]:
retw2v.Similarity(cosineSim1)

In [68]:
retw2v.getMap()

0.6743543764247651

# Adding synonyms using wordnet

In [1]:
from nltk.corpus import wordnet
import nltk
nltk.download('wordnet')

def addSynonymns(words):
  new_words = []
  for word in words:
    for syn in wordnet.synsets(word):
      for l in syn.lemmas():
        if l.name() not in new_words:
          new_words.append(l.name())
        
  return new_words

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [14]:
QWithSyns = {query:" ".join(addSynonymns(Queries[query].split())) for query in Queries} 

In [55]:
r = RochioAlgorithm(QWithSyns, edir, docs)

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [57]:
r.vectorizer(TfidfVectorizer)

In [58]:
r.getOptimizedQueries(r.tfidfvec, r.tfidfq, 1, 0.75, 0.15)

In [59]:
c1 = count() 

In [60]:
def isin(el, obj):
  for x in obj:
    if x[0] == el:
      return True
  return False

In [61]:
ret = Retrieval(r, r.tfidfvec)

In [ ]:
ret.Similarity(cosineSim)

In [66]:
ret.getMap()

0.6574325674258753

# Word2Vec


In [ ]:
files = getdocFiles()

In [ ]:
r.word2vecVectorizer(files)

In [ ]:
r.getOptimizedQueries(r.docsV, r.w2vQueries, 1, 0.75, 0.15)

In [ ]:
def count():
  i = 0 
  while True:
    yield i
    i+=1
c= count()
fileIndexer = {file:next(c) for file in files}

In [ ]:
retw2v = Retrieval(r, r.docsV)

In [ ]:
def cosineSim1(a, b):
  an = np.linalg.norm(a)
  bn = np.linalg.norm(b)
  if an == 0 or bn == 0:
    return 0

  return np.dot(a, b)/(an*bn)

In [ ]:
retw2v.Similarity(cosineSim1)

In [67]:
retw2v.getMap()

0.5467523645983254